# Practicas en empresa - Smart Rural

## Cargamos librerias

In [ ]:
import os
from ultralytics import YOLO
import albumentations as A
import numpy as np
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import math
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

## Data Augmentation

Creamos una funcion que nos crea

In [1]:
def augment_images_with_labels(image_folder, label_folder, num_augmented_images):
    transform = A.Compose(
        [A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.CLAHE(clip_limit=2, p=0.2),
        A.Transpose()],
        bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])
    )

    def load_labels(path):
        try:
            return np.loadtxt(path).reshape(-1, 5)
        except:
            return np.array([]).reshape(-1, 5)

    output_image_folder = os.path.join(image_folder, 'augmented')
    output_label_folder = os.path.join(label_folder, 'augmented')
    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_label_folder, exist_ok=True)

    for i in range(num_augmented_images):
        random_image_file = np.random.choice(os.listdir(image_folder))
        random_image_path = os.path.join(image_folder, random_image_file)
        image = cv2.imread(random_image_path)
        if image is None: 
            print(f"Image at {random_image_path} could not be read.")
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        corresponding_label_file = os.path.splitext(random_image_file)[0] + '.txt'
        label_path = os.path.join(label_folder, corresponding_label_file)

        labels = load_labels(label_path)
        if labels.size == 0:
            continue

        class_labels = labels[:, 0]
        boxes = labels[:, 1:]
        
        transformed = transform(image=image, bboxes=boxes, class_labels=class_labels)
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']
        transformed_class_labels = transformed['class_labels']

        output_image_path = os.path.join(output_image_folder, f'augmented_{i}.jpg')
        transformed_image = cv2.cvtColor(transformed_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_image_path, transformed_image)
        
        augmented_boxes = np.array(transformed_bboxes)
        augmented_class_labels = np.array(transformed_class_labels)
        if len(augmented_class_labels) != 0 and len(augmented_boxes) != 0:
            output_label_path = os.path.join(output_label_folder, f'augmented_{i}.txt')
            augmented_labels = np.column_stack((augmented_class_labels, augmented_boxes))
            np.savetxt(output_label_path, augmented_labels, fmt='%d %.6f %.6f %.6f %.6f')

    print(f"Las imagenes y las etiquetas han sido guardadas en {output_image_folder} y {output_label_folder} respectivamente.")

Indicamos donde queremos en que ruta estan las imagenes y los labels que queremos aumentar y cuantas imagenes queremos crear, en nuestro caso 100:

In [ ]:
augment_images_with_labels('datasets/data/images/val/', 'datasets/data/labels/val/', 100)

## Entrenamos el modelo de detección de uvas

Definimos la ruta donde estaran las carpetas con nuestras imagenes y el archivo de configuración para YOLOv8, este archivo de cocnfiguración 

In [ ]:
ROOT_DIR = '/datasets/data/'

Por temas de memoria se usa la version nano del modelo preentrenado de YOLOv8

In [ ]:
model = YOLO('yolov8n.pt')

Entrenamos el modelo

In [ ]:
results = model.train(data=os.path.join(ROOT_DIR,"config.yaml"), imgsz=800, augment=False, epochs = 100,optimizer='Adam', plots=True, batch=32, lr0 = 0.0001, weight_decay=0.0005, iou = 0.45)

## Entrenamos el modelo para la retección de QR

Entrenamos el modelo

In [ ]:
results = model.train(data=os.path.join(ROOT_DIR,"config_qr.yaml"), imgsz=800, augment=True, epochs = 200,optimizer='Adam', plots=True, batch=32, lr0 = 0.0001, iou = 0.4)

## Predecimos y calculamos áreas y tamaños

Creamos las funciones necesarias, la primera es para dividir la ruta y quedarnos con el nombre de la imagen la segunda nos calcula la dimension del QR y la tercerda dada la dimension del QR la dimension de las uvas

In [2]:
def nombre_imagen(path):
    nombre_imagen = path.split("\\")[-1]
    return nombre_imagen

In [8]:
def calcula_dim_qr(c_qr):
    if not c_qr:  # Comprueba si la lista está vacía
        return 0
    
    if isinstance(c_qr[0], list):  # Si c_qr es una lista de listas
        total_pixels_to_cm2_ratio = 0
        total_listas = len(c_qr)

        for lista in c_qr:
            ratio = calcula_dim_qr(lista)  # Llamada recursiva para cada lista individual
            total_pixels_to_cm2_ratio += ratio

        pixels_to_cm2_ratio = total_pixels_to_cm2_ratio / total_listas
    else:
        # Coordenadas xyxy de la caja delimitadora del cuadrado
        x1_square, y1_square, x2_square, y2_square = c_qr[0], c_qr[1], c_qr[2], c_qr[3]

        # Calcular el ancho y la altura de las cajas delimitadoras
        box_width_square = x2_square - x1_square
        box_height_square = y2_square - y1_square

        # Calcular las áreas de las cajas delimitadoras en píxeles
        area_square_pixels = box_width_square * box_height_square

        # Sabemos el area del qr que es 324 cm²
        area_square_real = 324 

        # Calcular la relación entre los píxeles y el tamaño real
        pixels_to_cm2_ratio = area_square_real / area_square_pixels
    return pixels_to_cm2_ratio


In [22]:
def calcula_dim_grape(c_grape, pixels_to_cm2_ratio):
    if not c_grape:  # Comprueba si la lista está vacía
        print("La lista está vacía.")
        return [[0,0,0]]

    if isinstance(c_grape[0], list):  # Si c_grape es una lista de listas
        results = []  # Lista para almacenar los resultados
        for lista in c_grape:
            if not lista:  # Comprueba si la sublista está vacía
                print("La sublista está vacía.")
                continue  # Salta esta sublista y pasa a la siguiente
            result = calcula_dim_grape(lista, pixels_to_cm2_ratio)  # Llamada recursiva para cada lista individual
            results.append(result)  # Almacena los resultados
        return results  # Devuelve la lista de resultados
    else:
        # Coordenadas xyxy de la caja delimitadora del racimo
        x1_obj, y1_obj, x2_obj, y2_obj = c_grape[0], c_grape[1], c_grape[2], c_grape[3]

        # Calcular el ancho y la altura de las cajas delimitadoras
        box_width_obj = x2_obj - x1_obj
        box_height_obj = y2_obj - y1_obj

        # Calcular las áreas de las cajas delimitadoras en píxeles
        area_obj_pixels = box_width_obj * box_height_obj

        # Calcular la relación entre los píxeles y el tamaño real (cm) para el cuadrado
        pixels_to_cm_ratio = math.sqrt(pixels_to_cm2_ratio)
    
        # Calcular el tamaño real del racimo
        area_obj_real = area_obj_pixels * pixels_to_cm2_ratio 

        # Calcular las dimensiones estimadas del racimo
        width_obj_real = box_width_obj * pixels_to_cm_ratio
        height_obj_real = box_height_obj * pixels_to_cm_ratio

        # print(f'El tamaño estimado del racimo es de {round(area_obj_real,2)} cm².')
        # print(f'Las dimensiones estimadas del racimo son {round(width_obj_real,2)} cm x {round(height_obj_real,2)} cm.')
        # print(area_obj_real, width_obj_real, height_obj_real)
        return area_obj_real, width_obj_real, height_obj_real


Cargamos los modelos que mejor pesos nos han dado en los entrenamientos:

In [10]:
model_qr = YOLO("runs/detect/trainQR/train/weights/best.pt")

In [11]:
model_grapes = YOLO("runs/detect/P42/weights/best.pt")

Predecimos con el dataset de test las uvas y los QR

In [18]:
results_qr = model_qr.predict(source="datasets/data/images/test", save=True, save_txt=True, iou=0.5, line_width=5, conf=0.15, show_labels=False)



image 1/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\images\test\1597909298344.jpg: 608x800 1 qr, 302.0ms
image 2/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\images\test\1597909980616.jpg: 800x608 2 qrs, 296.0ms
image 3/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\images\test\1597924845589.jpg: 608x800 1 qr, 316.6ms
image 4/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\images\test\1597994671291.jpg: 608x800 2 qrs, 298.0ms
image 5/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\images\test\1598001053941.jpg: 608x800 2 qrs, 295.0ms
image 6/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\images\test\1598001818948.jpg: 608x800 2 qrs, 276.7ms
image 7/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\images\test\1598341617511.jpg: 608x800 2 qrs, 283.0ms
image 8/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\images\test\1598342953878.jpg: 608x800 2 qrs, 2

In [19]:
results_grapes = model_grapes.predict(source="datasets/data/images/test", save=True, save_txt=True, iou=0.1, line_width=5, conf=0.2, show_labels=False)



image 1/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\images\test\1597909298344.jpg: 608x800 21 grapess, 1053.1ms
image 2/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\images\test\1597909980616.jpg: 800x608 24 grapess, 1029.1ms
image 3/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\images\test\1597924845589.jpg: 608x800 11 grapess, 1013.6ms
image 4/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\images\test\1597994671291.jpg: 608x800 10 grapess, 986.2ms
image 5/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\images\test\1598001053941.jpg: 608x800 26 grapess, 1023.7ms
image 6/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\images\test\1598001818948.jpg: 608x800 28 grapess, 1050.0ms
image 7/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\images\test\1598341617511.jpg: 608x800 14 grapess, 1020.0ms
image 8/14 C:\Users\administrador\Documents\MIA\Practicas\datasets\data\imag

Obtenemos los datos de los bounding boxes de los QR y de las Uvas y calculamos las dimensiones de las uvas dadas las de los QR

In [20]:
list_results = []
for i, (result_qr,result_grape) in enumerate(zip(results_qr,results_grapes)):
    name = nombre_imagen(result_qr.path)
    list_results.append([name])
    print(name)
    r = result_qr.boxes.xyxy.tolist()
    g = result_grape.boxes.xyxy.tolist()
    d = calcula_dim_qr(r)
    datos = calcula_dim_grape(g,calcula_dim_qr(r))
    for data in datos:
        area, width, height = data
        # print(area,width,height)
        list_results.append(list(data))
# print(list_results)
   



1597909298344.jpg
1597909980616.jpg
1597924845589.jpg
1597994671291.jpg
1598001053941.jpg
1598001818948.jpg
1598341617511.jpg
1598342953878.jpg
1599030766103.jpg
1599032188801.jpg
1602141746308.jpg
1606994183464.jpg
1606994877322.jpg
1606995956803.jpg


Procesamos los datos para darlos en forma de dataframe y descargarlos en excel, por cada imagen nos muestra las detecciones de uvas que ha hecho su area su ancho y su largo.

In [21]:
# Preprocesar los datos
data = []
identificador = []
conteo_por_id = {}
for line in list_results:
    if len(line) == 1:  # Es un ID
        identificador = line
        conteo_por_id[identificador[0]] = 0  # Inicializar el conteo para el ID
    else:  # Es un conjunto de valores
        conteo_por_id[identificador[0]] += 1  # Incrementar el conteo para el ID
        row =  identificador + [conteo_por_id[identificador[0]]] + line  # Añadimos el ID, los valores y el conteo
        data.append(row)

# Crear el dataframe
df = pd.DataFrame(data, columns=["ID", "Conteo", "Area", "Ancho", "Largo"])
print(df)
df.to_csv('total.csv',sep=';', decimal=',', index=False)


                    ID  Conteo        Area      Ancho      Largo
0    1597909298344.jpg       1  411.192880  17.277664  23.799101
1    1597909298344.jpg       2  309.115069  12.597283  24.538233
2    1597909298344.jpg       3  227.771083  11.850027  19.221144
3    1597909298344.jpg       4  442.067162  15.874196  27.848161
4    1597909298344.jpg       5  247.942293  15.164959  16.349684
..                 ...     ...         ...        ...        ...
303  1606995956803.jpg      14  346.757007  14.150853  24.504319
304  1606995956803.jpg      15   98.042591   9.474602  10.347938
305  1606995956803.jpg      16   72.106373   6.462968  11.156851
306  1606995956803.jpg      17   66.216752   7.425076   8.917990
307  1606995956803.jpg      18   67.916367   7.874709   8.624619

[308 rows x 5 columns]
